In [1]:
import tensorflow as tf
import numpy as np
import simplejson
import matplotlib.pyplot as plt
%matplotlib inline
import threading

import utils.train_utils as train_utils
import utils.googlenet_load as googlenet_load
import tensorflow.contrib.rnn as rnn
import time
import tensorflow.contrib.slim as slim
import string

In [2]:
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops

In [3]:
@ops.RegisterGradient('Hungarian')
def _hungarian_grad(op, *args):
    return map(array_ops.zeros_like, op.inputs)

In [4]:
def enqueue_thread(sess, coord, enqueue, gen, phase):
    while not coord.should_stop():
        data_gen = gen.next()
        image = data_gen['image']
        confs = data_gen['confs']
        boxes = data_gen['boxes']
        sess.run(enqueue, feed_dict={x_in[phase]:image, box_in[phase]:boxes, conf_in[phase]:confs})

In [5]:
def build_lstm(x, H, reuse):
    '''
    Building LSTM layers
    '''
    rnn_len = H['rnn_len']
    num_states = H['num_states']
    num_lstm_layers = H['num_lstm_layers']
    grid_height = H['grid_height']
    grid_width = H['grid_width']
    grid_size = grid_height*grid_width
    batch_size = H['batch_size']
    outer_size = batch_size*grid_size
    
    lstm_cells = rnn.MultiRNNCell(
        cells=[rnn.BasicLSTMCell(num_units=num_states,reuse=reuse) for _ in range(num_lstm_layers)])
    
    state = lstm_cells.zero_state(batch_size=outer_size, dtype=tf.float32)
    
    lstm_outputs = []
    
    initializer = tf.random_uniform_initializer(-0.1, 0.1)
    with tf.variable_scope('lstm', reuse=reuse, initializer=initializer):
        for i in range(rnn_len):
            if i>0: tf.get_variable_scope().reuse_variables()
            lstm_out, state = lstm_cells(x, state)
            lstm_outputs.append(lstm_out)
    return lstm_outputs

In [6]:
def build_pred(H, x_in, phase):
    '''
    Building the feedforward NN for both training and test
    '''
    grid_height = H['grid_height']
    grid_width = H['grid_width']
    grid_size = grid_height*grid_width
    batch_size = H['batch_size']
    outer_size = batch_size*grid_size
    later_feat_channels = H['later_feat_channels']
    num_states = H['num_states']

    batch_size = H['batch_size']
    
    reuse = {'train': False, 'test': True}[phase]
    is_training = {'train': True, 'test': False}[phase]

    x_in -= 117
    feat_5c, feat_3b = googlenet_load.model(x=x_in, is_training=is_training, H=H, reuse=reuse)

    scaling_factor = 0.01
    lstm_input = tf.reshape(feat_5c*scaling_factor, shape=[outer_size, later_feat_channels])

    lstm_outputs = build_lstm(lstm_input, H, reuse)

    initializer = tf.random_uniform_initializer(-0.1, 0.1)
    pred_boxes_c, pred_logits_c = [], []
    with tf.variable_scope('decoder', reuse=reuse, initializer=initializer):
        for i in range(rnn_len):
            box_weight = tf.get_variable(dtype=tf.float32,name='box_weight_%i' %i, shape=[num_states, 4])
            logit_weight = tf.get_variable(dtype=tf.float32,name='conf_weight_%i' %i, shape=[num_states, 2])
            pred_box = tf.matmul(lstm_outputs[i], box_weight)*50.
            pred_logit = tf.matmul(lstm_outputs[i], logit_weight)
            pred_box_r = tf.reshape(pred_box, shape=[outer_size,1,4])
            pred_logit_r = tf.reshape(pred_logit, shape=[outer_size,1,2])
            pred_boxes_c.append(pred_box_r)
            pred_logits_c.append(pred_logit_r)
        pred_boxes = tf.concat(values=pred_boxes_c, axis=1)
        pred_logits = tf.concat(values=pred_logits_c, axis=1)
        pred_confs = tf.nn.softmax(logits=pred_logits)
        
    return pred_boxes, pred_logits, pred_confs

In [7]:
def build_loss(H, q):
    '''
    Building the feedforward NN for both training and test, using build_pred
    And then:
    Building the loss function
    '''
    grid_height = H['grid_height']
    grid_width = H['grid_width']
    grid_size = grid_height*grid_width
    batch_size = H['batch_size']
    outer_size = batch_size*grid_size
    rnn_len = H['rnn_len']
    hungarian_iou = H['hungarian_iou']
    conf_weight = H['conf_weight']
    box_weight = H['box_weight']
    
    pred_boxes, pred_logits, pred_confs, loss_box, loss_conf, accuracy, loss = {}, {}, {}, {}, {}, {}, {}
    hungarian_module = tf.load_op_library('utils/hungarian/hungarian.so')

    for phase in ['train' ,'test']:
        x_in, box_in ,conf_in = q[phase].dequeue_many(batch_size)
        
        flag_in = tf.arg_max(conf_in, dimension=-1)
        
        box_r = tf.reshape(box_in, shape=[outer_size, rnn_len, 4])
        flag_r = tf.cast(tf.reshape(flag_in, shape=[outer_size, rnn_len]), tf.int32)

        pred_boxes[phase], pred_logits[phase], pred_confs[phase] = build_pred(H, x_in, phase)
        # classes is the reordered correct label
        # perm_truth is the reordered correct boxes
        # pred_mask is the mask for the predicted boxes
        assignments, classes, perm_truth, pred_mask = hungarian_module.hungarian(
            pred_boxes[phase], box_r, flag_r, hungarian_iou)
        true_class = tf.cast(tf.greater(classes, 0), dtype=tf.int32)
        loss_conf[phase] = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=pred_logits[phase], labels=true_class))*conf_weight
        residual = tf.abs(perm_truth-pred_boxes[phase]*pred_mask)
        loss_box[phase] = tf.reduce_mean(residual)*box_weight
        loss[phase] = loss_box[phase]+loss_conf[phase]
        pred_flag = tf.cast(tf.arg_max(pred_logits[phase], dimension=-1), tf.int32)
        accuracy[phase] = tf.reduce_mean(tf.cast(tf.equal(flag_r, pred_flag),tf.float32))
    return pred_boxes, pred_confs, loss, loss_box, loss_conf, accuracy

In [8]:
def build(H, q):
    '''
    Building the train_op, summary
    '''
    clip_norm = H['clip_norm']
    log_dir = 'logdir'
    pred_boxes, pred_confs, loss, loss_box, loss_conf, accuracy = build_loss(H, q)
    
    # train_op
    lr = tf.placeholder(tf.float32, shape=[])
    opt = tf.train.RMSPropOptimizer(decay=0.9,learning_rate=lr, epsilon=0.0001)
    tvars = tf.trainable_variables()
    grads = tf.gradients(loss['train'], tvars)
    grads, norm = tf.clip_by_global_norm(grads, clip_norm)
    global_step = tf.Variable(0, trainable=False)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = opt.apply_gradients(zip(grads, tvars), global_step=global_step)
    
    # building the summary
    for phase in ['train', 'test']:
        tf.summary.scalar('%s/loss'%phase, loss[phase])
        tf.summary.scalar('%s/accuracy'%phase, accuracy[phase])
    
    summary_op = tf.summary.merge_all()
    return train_op, summary_op, global_step, loss, accuracy,lr

In [16]:
hypes = 'hypes/lstm_cnn.json'
with open(hypes) as f:
    H = simplejson.load(f)
image_height = H['image_height']
image_width = H['image_width']
queue_size = H['queue_size']
rnn_len = H['rnn_len']
num_classes = H['num_classes']
grid_width = H['grid_width']
grid_height = H['grid_height']
grid_size = grid_height*grid_width
batch_size = H['batch_size']
clip_norm = H['clip_norm']
max_iter = H['max_iter']
display_iter = H['display_iter']
learning_rate = H['learning_rate']
lr_step = H['lr_step']
save_iter = H['save_iter']
log_dir = H['log_dir']

In [10]:
queue, enqueue_op, x_in, box_in, conf_in = {}, {}, {}, {}, {}
shape = ([image_height, image_width, 3],
         [grid_size, rnn_len, 4],
         [grid_size, rnn_len, num_classes]
        )
for phase in ['train', 'test']:
    x_in[phase] = tf.placeholder(tf.float32)
    box_in[phase] = tf.placeholder(tf.float32)
    conf_in[phase] = tf.placeholder(tf.float32)
    queue[phase] = tf.FIFOQueue(capacity=queue_size,
                                shapes=shape,
                                dtypes=(tf.float32, tf.float32, tf.float32)
                               )
    enqueue_op[phase] = queue[phase].enqueue((x_in[phase], box_in[phase], conf_in[phase]))

In [11]:
train_op, summary_op, global_step, loss, accuracy,lr = build(H, queue)

In [13]:
for ops in tf.trainable_variables(): print ops.name

InceptionV1/Conv2d_1a_7x7/weights:0
InceptionV1/Conv2d_1a_7x7/BatchNorm/beta:0
InceptionV1/Conv2d_1a_7x7/BatchNorm/gamma:0
InceptionV1/Conv2d_2b_1x1/weights:0
InceptionV1/Conv2d_2b_1x1/BatchNorm/beta:0
InceptionV1/Conv2d_2b_1x1/BatchNorm/gamma:0
InceptionV1/Conv2d_2c_3x3/weights:0
InceptionV1/Conv2d_2c_3x3/BatchNorm/beta:0
InceptionV1/Conv2d_2c_3x3/BatchNorm/gamma:0
InceptionV1/Mixed_3b/Branch_0/Conv2d_0a_1x1/weights:0
InceptionV1/Mixed_3b/Branch_0/Conv2d_0a_1x1/BatchNorm/beta:0
InceptionV1/Mixed_3b/Branch_0/Conv2d_0a_1x1/BatchNorm/gamma:0
InceptionV1/Mixed_3b/Branch_1/Conv2d_0a_1x1/weights:0
InceptionV1/Mixed_3b/Branch_1/Conv2d_0a_1x1/BatchNorm/beta:0
InceptionV1/Mixed_3b/Branch_1/Conv2d_0a_1x1/BatchNorm/gamma:0
InceptionV1/Mixed_3b/Branch_1/Conv2d_0b_3x3/weights:0
InceptionV1/Mixed_3b/Branch_1/Conv2d_0b_3x3/BatchNorm/beta:0
InceptionV1/Mixed_3b/Branch_1/Conv2d_0b_3x3/BatchNorm/gamma:0
InceptionV1/Mixed_3b/Branch_2/Conv2d_0a_1x1/weights:0
InceptionV1/Mixed_3b/Branch_2/Conv2d_0a_1x1/Ba

In [ ]:
summary_writer = tf.summary.FileWriter(logdir=log_dir+'/summary/', flush_secs=10)
saver = tf.train.Saver(max_to_keep=None)

with tf.Session() as sess:
    t, gen = {}, {}
    coord = tf.train.Coordinator()
    for phase in ['train', 'test']:
        gen[phase] = train_utils.load_data_gen(H=H, jitter=False, phase=phase)
        t[phase] = threading.Thread(target=enqueue_thread, 
                                    args=(sess, coord, enqueue_op[phase], gen[phase], phase))
        t[phase].start()
    summary_writer.add_graph(sess.graph)
    start = time.time()
    sess.run(tf.global_variables_initializer())
    init_fn = slim.assign_from_checkpoint_fn(
        ignore_missing_vars=True,
        model_path='data/inception_v1.ckpt', 
        var_list=slim.get_model_variables('InceptionV1'))
    init_fn(sess)
    for i in range(max_iter):
        adjusted_lr = learning_rate*0.5**max(0,(np.int32(i/lr_step)-2))
        if i%display_iter != 0:
            sess.run([train_op, loss['train']], feed_dict={lr: adjusted_lr})
        else:
            if i>0:
                dt = (time.time()-start)/batch_size/display_iter
            start = time.time()
            train_loss, test_accuracy, summaries, _ = sess.run(
                [loss['train'], accuracy['test'], summary_op, train_op], feed_dict={lr: adjusted_lr})
            summary_writer.add_summary(summaries, global_step=global_step.eval())
            print_str = string.join([
                'Step: %d',
                'lr: %f',
                'Train loss: %.2f',
                'Test accuracy: %.1f',
                'Time/image (ms) %.1f'
            ])
            print(print_str %(i, adjusted_lr, train_loss, test_accuracy*100, dt*1000 if i>0 else 0))
        if global_step.eval() % save_iter ==0 or global_step.eval() == max_iter-1:
            saver.save(sess, log_dir+'/checkpoint/', global_step=global_step.eval())

INFO:tensorflow:Restoring parameters from data/inception_v1.ckpt
Step: 0 lr: 0.001000 Train loss: 0.72 Test accuracy: 35.5 Time/image (ms) 0.0
Step: 50 lr: 0.001000 Train loss: 0.71 Test accuracy: 97.2 Time/image (ms) 3550.6
Step: 100 lr: 0.001000 Train loss: 0.54 Test accuracy: 99.0 Time/image (ms) 3298.3
Step: 150 lr: 0.001000 Train loss: 0.28 Test accuracy: 99.0 Time/image (ms) 29147.9
Step: 200 lr: 0.001000 Train loss: 0.20 Test accuracy: 99.3 Time/image (ms) 3571.2
Step: 250 lr: 0.001000 Train loss: 0.16 Test accuracy: 96.9 Time/image (ms) 3618.1
Step: 300 lr: 0.001000 Train loss: 0.12 Test accuracy: 97.8 Time/image (ms) 3480.7
Step: 350 lr: 0.001000 Train loss: 0.12 Test accuracy: 99.4 Time/image (ms) 3314.3
Step: 400 lr: 0.001000 Train loss: 0.07 Test accuracy: 98.0 Time/image (ms) 44961.6
Step: 450 lr: 0.001000 Train loss: 0.06 Test accuracy: 99.1 Time/image (ms) 10118.3
Step: 500 lr: 0.001000 Train loss: 0.05 Test accuracy: 98.2 Time/image (ms) 3343.0
Step: 550 lr: 0.001000 Tr

In [ ]:
coord.request_stop()
coord.join()

In [ ]:
xx, yy, zz = sess.run(queue['train'].dequeue())

In [ ]:
xx.shape

In [ ]:
plt.imshow(xx)

In [ ]:
np.int32(5.7)